# Variational Auto Encoder

VAE, 変分オートエンコーダ{cite}`VAE`

今回はVAEというAEの亜種について，BoWに対する言語モデル（BoWで表現された文書を生成するようなVAE）を例に紹介します．

:::{note}
本題に入る前に...   
AEではencoderの出力をそのまま入力ベクトルの潜在ベクトル（圧縮表現）として利用していましたね．これに対してVAEは，潜在ベクトルの各要素を正規分布からのサンプルの一つだと仮定します．

そのためencoderの出力はそのまま使わず，encoderはデータからガウス分布のパラメータ（平均と標準偏差）を作成するために利用されます．画像データなどをVAEで生成する場合はまさにこのままの考え方でOKです．

ですがここではもう少し進んで，文書データの生成過程をVAEを使って再現してみます．文書データと言ってもBoW (Bag-of-Words）なので語順などは無視して単語の出現頻度だけを記録した行列です．

さて，もしも全世界にある文書を全て集めてこれたのならば：  
1. ある文書の潜在ベクトルは __標準正規分布__ からのサンプリングによって生成されていて，
2. その潜在ベクトルを適当な関数（ニューラルネット）でそれぞれの文書用の単語頻度分布（全ての要素が0以上で合計が1のベクトル）に変換して，
3. この単語頻度分布に従う離散確率分布からのサンプリングによって文書の中の単語がそれぞれ生成される

という流れで文書ができていると考えます．

私たちが今持っている文書群はこの母集団からサンプリングしてきた一部に過ぎないのです．そのため，実際に扱うデータの潜在ベクトル（の各要素）は標準正規分布に近いけど少し違うガウス分布に従っていることが想像できます．


:::


In [1]:
import os
import sys

from collections import OrderedDict
from copy import deepcopy
from time import time

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [2]:
m = 128 # minibatch size
v = 1000 # number of features. in this case, vocaburary size
k = 20 # latent dimension size.

x = np.random.random([1,v])
h = np.random.random([1, k])
W = np.random.random([k, v])
b = np.random.random(v)

x_hat = h@W +b
print(f"{x_hat.shape=}")
reconstruction_error = ((x - x_hat)**2).sum(axis=1).mean()
print(f"{reconstruction_error=}")

x_hat.shape=(1, 1000)
reconstruction_error=36603.369294606615


::::{note}  
ちょっと脱線：文書群からのトピック抽出

まずは三層のAEについて思い出しながら考えていきましょう．

![](https://cdn-ak.f.st-hatena.com/images/fotolife/n/nkdkccmbr/20161006/20161006215630.png)  
オートエンコーダのイメージ  
出典：[オートエンコーダ-- ニューラルネットワーク・DeepLearningなどの画像素材　プレゼン・ゼミなどに【WTFPL】](https://nkdkccmbr.hateblo.jp/entry/2016/10/06/222245)

一つの文書のBoWベクトル$\mathbf{x}\in \mathbb{R}^{v}$が入力されたEncoderは，これを潜在空間上の点として見ることができるベクトル$\mathbf{h}\in \mathbb{R}^{k}$ へ符号化します．
Decoderの活性化関数が恒等関数だとすると， $\hat{\mathbf{x}}=\mathbf{h} \cdot \mathbf{W} + \mathbf{b}$を行なっているわけです．

```python
# In [1]:
m = 128 # minibatch size
v = 1000 # number of features. in this case, vocaburary size
k = 20 # latent dimension size.

x = np.random.random([1,v])
h = np.random.random([1, k])
W = np.random.random([k, v])
b = np.random.random(v)

x_hat = h@W +b
print(f"{x_hat.shape=}")
reconstruction_error = ((x - x_hat)**2).sum(axis=1).mean()
print(f"{reconstruction_error=}")

# Out [1]:
# x_hat.shape=(1, 1000)
# reconstruction_error=19712.92650866515
```




ミニバッチごとに処理されるとすると，元行列$\mathbf{X} \in \mathbb{R}^{m\times v}$,再構成行列$\hat{\mathbf{X}} \in \mathbb{R}^{m\times v}$, 潜在空間にマップされた行列$\mathbf{H}\in \mathbb{R}^{m\times k}$, デコーダの重み行列$\mathbf{W} \in \mathbf{R}^{k\times v}$であり，biasを無視するとこれはほぼ$\hat{\mathbf{X}}=\mathbf{H} \cdot \mathbf{W}$と表せます．再構成行列は元行列に近くなるように訓練するので，このオートエンコーダは元の行列を二つの行列$\mathbf{H}$と$\mathbf{W}$に行列分解していると見ることができます．

ではここで出てきた$k$とはなんでしょうか？オートエンコーダにおけるボトルネックレイヤの次元数のことなのですが，文書データの行列分解の文脈で見るとどういう意味になりそうですか？

これをその文書群の中から見つけた __潜在的なトピック(話題)__ であるとする考え方があります．そうするとつまり，
- $\mathbf{H}$ は文書を話題の濃淡で表現した行列
- $\mathbf{W}$ は語彙を話題の濃淡で表現した行列

だということもできそうです．

例えば$k=3$として，それぞれのトピックが「スポーツ，政治，経済」に関するものであるとしましょう．ある文書の潜在ベクトルは，その文書が三つのトピックそれぞれにどれだけ属しているのかを表す値を持っていると言えそうです．

（もちろんencoderの最終層の活性化関数によっては，潜在変数は負の値を持っている場合があります．この場合は理解しづらいのですが...）

また，この時の$\mathbf{W}$は
- 1行目がスポーツトピックの持っている単語分布
- 2行目が政治トピックの持っている単語分布
- 3行目が経済トピックの持っている単語分布

だと見ることができそうです．（確率分布のように正規化されていないですが...）  

実はそれぞれのトピックの単語分布の中で，大きい値を持つ単語を10個程度ピックアップすれば，そのトピックがどんなトピックなのかが大体わかります．


このような方法を使ってトピックを見つけるタスクを __トピック抽出 （Topic Extraction)__ と呼びます．
::::


## データの準備

20newsgroupsを利用します．以下のようにbowを作成します．

In [3]:
# The number of words in the vocabulary
n_words = 1000

import re

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=("headers", "footers", "quotes"), )
data_samples = dataset.data
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.7, min_df=10, max_features=n_words, stop_words="english", strip_accents="ascii", )

t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 24.573s.
Extracting tf features for LDA...
done in 3.737s.


In [4]:
word2id = tf_vectorizer.vocabulary_
# print("Number of features = {}".format(len(word2id)))
# out[1]
# Number of features = 1000
id2word = list(word2id.keys())
# print("First 20 features = {}".format(id2word[:20]))
# Out[2]
# First 20 features = ['sure', 'story', 'did', 'statement', 'media', 'pro', 'israeli', 'world', 'having', 'letter', 'try', 'think', 'reason', 'report', 'clearly', 'reports', 'received', 'government', 'makes', 'away']

for count, word in enumerate(word2id):
    if count < 10:
        print(word, word2id[word])

sure 873
story 861
did 301
statement 856
media 590
pro 715
israeli 497
world 985
having 442
letter 535


In [5]:
n_samples_tr = 10000
n_samples_te = tf.shape[0] - n_samples_tr
docs_tr = tf[:n_samples_tr, :]
docs_te = tf[n_samples_tr:, :]
print("Number of docs for training = {}".format(docs_tr.shape[0]))
print("Number of docs for test = {}".format(docs_te.shape[0]))

n_tokens = np.sum(docs_tr[docs_tr.nonzero()])
print(f"Number of tokens in training set = {n_tokens}")
print(
    "Sparsity = {}".format(len(docs_tr.nonzero()[0]) / float(docs_tr.shape[0] * docs_tr.shape[1]))
)

Number of docs for training = 10000
Number of docs for test = 1314
Number of tokens in training set = 479936
Sparsity = 0.0254311


In [6]:
docs_tr

<10000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 254311 stored elements in Compressed Sparse Row format>

ちなみに10000x1000の要素を持つ普通の配列が(dtypeがfloat32の場合）どのくらいのメモリを食うのかを確認してみると：

In [7]:
docs_te.toarray() # こうすれば元の配列に戻せる

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
K = 1024
byte = 8

In [9]:
(10000*1000*64)/(K*K*8)

76.2939453125

In [10]:
b.nbytes / (K*K)

0.00762939453125

そして疎行列型のdocs_trがどのくらいメモリを食っているのか:

```
Type:        csr_matrix
Docstring:  
Compressed Sparse Row matrix
...

Attributes
----------
...
data
    CSR format data array of the matrix
indices
    CSR format index array of the matrix
indptr
    CSR format index pointer array of the matrix
```

In [11]:
a = (
    docs_tr.indices.nbytes +
    docs_tr.indptr.nbytes +
    docs_tr.data.astype(np.float32).nbytes
    ) / K
print(a, "KB")

2025.87109375 KB


In [12]:
docs_tr.nnz*64 / (K*byte)

1986.8046875

## Auto Encoderからstep-by-stepでVAEを構築

VAEの気持ち：  
潜在ベクトルの各要素が標準正規分布に従って生成されていると仮定したAuto Encoder．


### 単純なAE

入力される特徴量が1000，　隠れ層の次元数が20のオートエンコーダクラスを書いてください．
- Encoderの最終層の活性化関数はなんでも良い
- Decoderの最終層はLinearとする

In [13]:
class AutoEncoder(nn.Module):
    def __init__(self, in_features:int=1000, n_components:int=20):
        super().__init__()
        self.in_features = in_features
        self.n_components = n_components
        # build layers
        self.encoder = nn.Sequential(
            nn.Linear(self.in_features, self.n_components*2),
            nn.Sigmoid(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.n_components, self.in_features),
        )

    def forward(self, x:torch.Tensor):
        encoded = self.encoder(x)
        return encoded, self.decoder(encoded)


BoWを単純に圧縮した後に再構成するならば，上のようなAEでOKです．

ではここで出力される$\hat{x}$に着目して，その要素一つ一つがこのコーパスに登場した語彙（ユニークな単語）に対応していることを思い出してください．

例えば登場する語彙が以下の6個だけだったします．
1. Happy
1. Life
1. Home
1. TamaHome
1. Unhappy
1. House

ある文書でそれぞれの単語が出現するかどうかを1~6の目があるサイコロを何回か転がす試行で決めるとしましょう．その文書が実は"Happy Life Happy Home TamaHome"というものだったら， この中に登場した単語（トークン）の数は以下の5つになります．

1. Happy
2. Life
3. Happy
4. Home
5. TamaHome


よって6回サイコロを転がせば（うまく行けば）元の文書と同じ単語リストを獲得できそうです．

うまく元の単語リストと同じものを元の文書の単語数と同じ5回だけのサンプリングで出すためには，この文書用のサイコロのそれぞれの面を，単語の出現回数に合わせて出やすさを調節してあげる（広くしたり狭くしたりと歪に歪ませる）必要があります．

このような歪なサイコロを一回転がすようなサンプリングをするためには，カテゴリカル分布という確率分布を利用します．また複数回転がす場合は，多項分布と呼ばれる確率分布を利用します．

そしてこれらの確率分布の持つサイコロの歪さを表現するためには，全ての面の出やすさを持ったベクトルが必要です．このベクトルの各要素は簡単にいうと面の広さや重さに相当するので，0より大きくて1より小さい値になります．そして全ての要素の合計が1になるように調節（正則化）しておく必要があります．

このような条件のベクトル作成するのにちょうどいい関数として，私たちはSoftmax関数を知っていますね．

それでは次は，先ほどのAEの出力層をSoftmaxにして，語彙数の分だけの面を持った歪なサイコロを再現するAuto Encoderを作成しましょう．


In [14]:
class AutoEncoder(nn.Module):
    def __init__(self, in_features:int=1000, n_components:int=20):
        super().__init__()
        self.in_features = in_features
        self.n_components = n_components
        # build layers
        self.encoder = nn.Sequential(
            nn.Linear(self.in_features, self.n_components),
            nn.Sigmoid(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.n_components, self.in_features),
            nn.Softmax(dim=1),
        )

    def forward(self, x:torch.Tensor):
        encoded = self.encoder(x)
        return encoded, self.decoder(encoded)

ae = AutoEncoder()
display(ae)
x = torch.from_numpy(docs_tr[:32].toarray().astype(np.float32))
encoded, decoded = ae(x)
encoded.shape, decoded.shape

AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=1000, out_features=20, bias=True)
    (1): Sigmoid()
  )
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=1000, bias=True)
    (1): Softmax(dim=1)
  )
)

(torch.Size([32, 20]), torch.Size([32, 1000]))

このAuto Encoderを訓練することで，それぞれの文書に対応した歪なサイコロを作成することができそうです．

歪なサイコロはその文書を圧縮したベクトルを入力することで，decoderで作成することができるんですね．

### 潜在ベクトルが正規分布から生成されていると仮定

上で文書は多項分布から生成されると仮定しました．ではこれまで圧縮ベクトルや潜在ベクトルと呼んでいたものは，どんな分布から生成されると仮定できるでしょうか？


::::{note}
ちょっと脱線：統計学的な文脈での __モデル__ と __モデリング__

wip

::::

VAEではこの潜在ベクトルの各要素が正規分布からサンプルされていると仮定します．正規分布のパラメータは平均$\mu$と標準偏差$\sigma$なので，これをencoderに生成させましょう．
潜在ベクトルの要素を$\mathbf{z} \sim \mathcal{N}(\mathbf{\mu}, \mathbf{\sigma}）$で生成するようにAutoEncoderを書き換え， VariationalAutoEncoderクラスを作ってください．ただし， ミニバッチ学習させると仮定すると， muとsigmaはそれぞれがバッチサイズ$\times$潜在次元数 の行列になります．

::::{note}
encoderの出力層は活性化関数なしのlinearにしましょう．これを推奨しているのは，tanhなど使ってmuやsigmaの取り得る範囲を限定しないためです．しかしながら，linearの出力をそのままsigmaとして使うと以下のようなエラーが発生します．
```python
batch_size = 8
latent_dim = 5
mu = torch.randn([batch_size, latent_dim])
sigma = torch.randn([batch_size, latent_dim])
z = torch.normal(mu, sigma)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-17-1b89a1aa8b20> in <cell line: 5>()
      3 mu = torch.randn([batch_size, latent_dim])
      4 sigma = torch.randn([batch_size, latent_dim])
----> 5 z = torch.normal(mu, sigma)

RuntimeError: normal expects all elements of std >= 0.0
```

muはともかく，sigmaは標準偏差です．これが0未満になることはあり得ないので，encoderが直接sigmaを出力するように設計するのは無理があります．（もしかしたら上手い乱数でエラーにならない可能性もないわけではないですが...）

そのため，encoderが直接に$\sigma$を吐いていると仮定せずに， __対数分散__ を吐いていると仮定しましょう．

:::{hint}  
std（標準偏差）を求める行については，以下の等式を参考にしてください．

$$
e^{0.5 \log (v a r)}=e^{\log \left(v a r^{0.5}\right)}=var^{0.5}=\sigma
$$

ref: [reparameterization trick in VAEs, How should we do this?](https://stats.stackexchange.com/questions/486158/reparameterization-trick-in-vaes-how-should-we-do-this)

- 標準偏差: $\sigma$  
- 分散: $var = $\sigma^2$  

- これはつまり: $var^{0.5} = \sigma$
- expとlogは打ち消しあう

:::  
::::

また，潜在ベクトルやencoder出力も見てみたいので，forwardメソッドの出力は，z, decoderの出力, mu, logvarにしましょう．

[問題] AutoEncoderクラスを参考に，encoderが正規分布のパラメータを出力し，torch.normal関数で潜在変数ベクトルzを生成するようなVAEクラスを作成してください．

::::{margin} Implementation tips
実装方法としては以下の二つのどちらかになるでしょう．
- encoderの最終層を二股にする
- 2倍のユニット数を持つレイヤにしてから出力を2つに分割する

::::

In [15]:
class VariationalAutoEncoder(nn.Module):
    def __init__(self, in_features:int=1000, n_components:int=20):
        super().__init__()
        self.in_features = in_features
        self.n_components = n_components
        # build layers
        self.encoder = nn.Sequential(
            nn.Linear(self.in_features, self.n_components),
            nn.Sigmoid(),
            nn.Linear(self.n_components, self.n_components*2),
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.n_components, self.in_features),
            nn.Softmax(dim=1),
        )

    def forward(self, x:torch.Tensor):
        encoded = self.encoder(x)
        mu = encoded[:, :self.n_components]
        logvar = encoded[:, self.n_components:]
        z = torch.normal(mu, torch.exp(logvar*0.5))
        return z, self.decoder(z), mu, logvar

ae = VariationalAutoEncoder()
display(ae)
x = torch.from_numpy(docs_tr[:32].toarray().astype(np.float32))
z, decoded, mu, logvar = ae(x)
z.shape, decoded.shape

VariationalAutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=1000, out_features=20, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=20, out_features=40, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=1000, bias=True)
    (1): Softmax(dim=1)
  )
)

(torch.Size([32, 20]), torch.Size([32, 1000]))

### Reparameterization Trick: サンプリングが挟まっても微分可能な形へ

さて，正規分布をそのまま使うとbachpropagationでパラメータ更新ができるというNeural Netsの利点がなくなってしまいます．（サンプリングは微分できないよ）

そのためこれを微分可能な操作で置き換える必要があります．

視点を変えると，標準正規分布の平均をmuだけズラして，sigmaだけツリガネの幅を膨よかにしてあげれば任意の平均と標準偏差を持った正規分布が作れるはずです．
この場合でも標準正規分布からのサンプリングが発生していますが，このサンプリングされた値を，エンコーダの入力値と同様に固定値だと考えれば，これ以外のmuとsigmaは微分可能な関数で作成されているので，ネットワーク全体も微分可能であり，backpropagation可能です．

これを __reparameterization trick（再パラメータ化トリック）__ と呼びます．

```python
def reparameterization_trick(mu, logvar):
    #z = Normal(mu,var)
    std = (logvar*0.5).exp()
    eps = torch.rand_like(mu)
    z = eps * std + mu
    return z
```

[問題] これをVAEのメソッドとして実装し，サンプリングの代わりに使用するVAEクラスを作成してください．
- ただし，訓練時にはreparameterization_trickを利用したサンプリングを行い，評価時にはmuをそのままzとして返すように工夫してください．

In [16]:
class VariationalAutoEncoder(nn.Module):
    def __init__(self, in_features:int=1000, n_components:int=20):
        super().__init__()
        self.in_features = in_features
        self.n_components = n_components
        # build layers
        self.encoder = nn.Sequential(
            nn.Linear(self.in_features, self.n_components),
            nn.Sigmoid(),
            nn.Linear(self.n_components, self.n_components*2),
        )
        self.decoder = nn.Sequential(
            nn.Linear(self.n_components, self.in_features),
            nn.Softmax(dim=1),
        )

    def forward(self, x:torch.Tensor):
        encoded = self.encoder(x)
        mu = encoded[:, :self.n_components]
        logvar = encoded[:, self.n_components:]
        z = self.reparameterization_trick(mu, logvar)
        return encoded, self.decoder(z), mu, logvar

    def reparameterization_trick(self, mu, logvar):
        #z = Normal(mu,logvar)
        #return z
        if self.training:
            std = (logvar*0.5).exp()
            eps = torch.rand_like(mu)
            z = eps * std + mu
        else:
            z = mu
        return z


ae = VariationalAutoEncoder()
display(ae)
x = torch.from_numpy(docs_tr[:32].toarray().astype(np.float32))
latent_vector, p,*_ = ae(x)
latent_vector.shape, p.shape

VariationalAutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=1000, out_features=20, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=20, out_features=40, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=1000, bias=True)
    (1): Softmax(dim=1)
  )
)

(torch.Size([32, 40]), torch.Size([32, 1000]))

これで潜在ベクトルと文書ベクトルのそれぞれに対して確率分布を仮定することができました．

## 損失関数

:::{note}  
最終的な形：  
ELBO = Cross Entropy + KL Divergence

Cross Entropyはほとんど再構成誤差，KL Divergenceは正則化項の役割を行っているとも取れる．
:::


### Cross Entropy



今回のサンプルデータはテキストデータであり，これのBoWを再構成するようなオートエンコーダをVAEで作るのが今回の目的でした．

VAEの最終層の活性化関数を恒等関数（要はLinearの後に活性化関数を利用しない場合）にすれば，BoWを本当に再構成させることは可能かもしれません．

しかしここまでで作成したVAEは，ネットワークの出力層（decoderの最終層）の活性化関数としてsoftmaxを利用しています．これは，ある文書がBoW vectorとして入力された時に全ての語彙に対して「その語彙がこの文書の中で出現する確率」を予測する問題にしたいからでした．

:::{note}  
単語の出現頻度はZipf則に従うことが知られています．

> ジップの法則（ジップのほうそく、Zipf's law）あるいはジフの法則とは、出現頻度が k 番目に大きい要素が全体に占める割合がに比例するという経験則

:::

Softmaxを損失関数として利用しているので，損失関数は二乗和誤差のままではいけません．　このVAEは，言ってみればBoWで利用している語彙をクラスとして見ていて，文書をそれらに対応するようにクラス分類しているので，ここでは __Cross Entropy__ を利用します．

In [17]:
# データの準備（仮のデータを生成）
vocab_size = 1000
n_docs = 1000
docs_tr = np.random.rand(n_docs, vocab_size).astype(np.float32)
docs_tr = torch.from_numpy(docs_tr)


[問題 h-1] BoWと，VAEの予測した単語出現確率を受け取ってCross Entropyを計算する関数を作成してください．

::::{margin} Implementation tips

cross entropyはデータごとの値が欲しいです．しかし実際にはバッチ学習を行いますので，この関数の返り値はバッチ平均を取ることになリます．  
ex: batch_sizeが1でも32でもより大きな値でも，1つあたりのcross entropyと同じような大きさの値を出力させます．

::::

In [18]:
def cross_entropy(pred_proba, bow):
    return - (torch.log(pred_proba)*bow).sum(1).mean()
cross_entropy(p, x)

tensor(803.1685, grad_fn=<NegBackward0>)

このCross EntropyはBoWの配列を参照し，element-wiseに負の対数尤度を単語数倍した値を返します．これを文書ごと（データごと）に合計し，これをその文書のcross entropyとしています．また，batch_size分だけ一気にデータが入力されるはずなので，batch_sizeで平均を取っています．

Cross Entropyが小さくなるということは，元のコーパスにおいて出現頻度の大きかった単語が大きな出現確率を持っており，逆に出現頻度が小さかった単語が小さな出現確率になるということです．

### Kullback-Leibler divergence


カルバック・ライブラー（リーブラ） ダイバージェンス

Cross Entropyだけを利用しても学習はできますが，VAEでは潜在変数が正規分布からサンプリングされていました．現状ではこれに対してなんの制約もつけていません．例えば元の入力データに対してノイズを付与し，それを取り除くようにオートエンコーダを学習させるものをDenoising Auto Encoderと呼びますが，現状はそれに近い形になっています．

VAEでは潜在変数の事前分布を標準正規分布だと仮定しているのがキモになっているので，これを活かしながら何かしらの正則化を行い，この潜在変数が実際に取る値に制限をかけることを考えます．


> 正則化は、学習時に用いる式に項を追加することによってとりうる重みの値の範囲を制限し、過度に重みが訓練データに対してのみ調整される（過学習する）ことを防ぐ役割を果たします。   
> [正則化とは – 【AI・機械学習用語集】](https://zero2one.jp/ai-word/regularization/#:~:text=%E6%AD%A3%E5%89%87%E5%8C%96%E3%81%AF%E3%80%81%E5%AD%A6%E7%BF%92%E6%99%82%E3%81%AB,%E9%98%B2%E3%81%90%E5%BD%B9%E5%89%B2%E3%82%92%E6%9E%9C%E3%81%9F%E3%81%97%E3%81%BE%E3%81%99%E3%80%82&text=%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%81%AB%E3%81%8A%E3%81%84%E3%81%A6%E9%81%8E%E5%AD%A6%E7%BF%92,%E9%81%B8%E6%8A%9E%E8%82%A2%E3%82%92%EF%BC%91%E3%81%A4%E9%81%B8%E3%81%B9%E3%80%82)

潜在ベクトルの真の母集団は標準正規分布に従っていると仮定しましたが，実際に私たちが集めてきたデータはそのサブセットにすぎません．ということはデータに偏りがあるはずなので，綺麗な標準正規分布に従っているとは思えません．そのため実際には，潜在変数（潜在ベクトル$\mathbf{z}$のそれぞれの要素）が独立したガウス分布に従っているだろうと我々は仮定したのでした．母集団が標準正規分布に従っているのですから，そのサブセットである訓練データの潜在変数もここから大きく離れたガウス分布に従っているとは思えませんので，平均と標準偏差が大きくなりすぎて必要以上にガウス分布が大きくなりすぎるのを抑制したくなります．

そこで標準正規分布とencoderの構築した正規分布とが似たような形になるように正則化を行います．つまり標準正規分布から離れれば離れるほどに罰則を与える（値が大きくなる）ような関数を導入します．これに二つの確率分布の疑似距離を求める __Kullback-Leibler divergence　(KLダイバージェンス)__ を使います．KLダイバージェンスは確率分布Pと確率分布Qとがどれだけ似ているのかを測ります．

$$
D_{K L}(P \| Q)=\int_{-\infty}^{\infty} p(x) \log \frac{p(x)}{q(x)} d x
$$

ここでPとQが全く同じであれば0を取り，違う形であればあるほど値は大きくなります．

また，Qが標準正規分布であるならば，式を整理して単純にできます．

encoderが作成したガウス分布と標準正規分布とのKLダイバージェンスを求める式：

$$
D_{KL}[N(\mu,\sigma)||N(0,1)] = -\frac{1}{2}\sum(1+\log(\sigma^2)-\mu^2-\sigma^2)
$$



[問題 h-2] 上記のKLダイバージェンスの式をPythonで実装してください．ただし，文書（データ）ごとにこの値を求めた上で，バッチ平均を出力します．

In [19]:
def kld(mu,sigma):
    kl = -0.5 * torch.sum(1 + torch.log(sigma**2) - mu**2 - sigma**2)
    return kl

In [20]:
_, _, mu, logvar = ae(x)
kl_loss = kld(mu, logvar)
print(f"KL Divergence: {kl_loss.item()}")

KL Divergence: 985.9890747070312


encoderの作るガウス分布は標準正規分布に近くなるように矯正していくわけですが，全く標準正規分布と同じようになってしまっても困ります．これはつまり`mu=torch.zeros([m,k]), sigma=torch(torch.ones([m,k]))`なので，encoderの出力がどんなデータが入ってきても同じになってしまいます．つまりencoderの影響が全くない状態になるわけです．ここまでやってしまうと困ってしまうので，実際にはKLダイバージェンスが0に近いけど完全に0ではない状態で学習が止まってくれればちょうどいいわけです．

###  Evidence Lower BOund (ELBO, 変分下限)

文書ごとの対数尤度から，先ほど説明したKLダイバージェンスを引いた値を __ELBO__ と呼びます．VAEではこれを最大化するようにパラメータを更新します．ただし，ニューラルネットワークの慣例に倣って最小化問題にしたいので，それぞれの項にマイナスをかけたNegative ELBOを実際の目的関数として利用します．

目的関数

$$
\text{Negative ELBO} = \operatorname{Cross Entropy}(p_{vae}(\hat{x} | \hat{z}), p(x)) + \operatorname{KL Divergence}(\mathcal{N}(0,1) || \mathcal{N}_{vae}(\hat{\mu}, \hat{\sigma}))
$$

## メトリクス

学習中や学習後のモデルの評価を行う関数をメトリクスと呼びます．

### Perplexity

__Perplexity__ は言語モデルの性能（予測精度）を評価するための指標の一つです．一般的な言語モデルだと「This is a __ 」のように`This is a`までを生成したら，その次の単語を予測するような処理を行っています．この時， __ に入る単語の候補の数は，言語モデルが賢くなればなるほど数が減っていくはずです． この次の単語の選択肢の数の平均を Perplexity と呼びます．

Perplexityは

$$ PPL = 2^{(\text{1単語あたりのエントロピー})} $$

で求められます．また通常， Perplexity はテストセットに対して求めます．

今回，「1単語あたりのエントロピー」に相当するのは，一単語あたりのクロスエントロピーです． 現在は一つの文章あたりの値を出しているので，少し変えてバッチ内の単語数で平均をとるようにすればOKです．

また，上の定義では2の冪乗になっていますが，計算する際にはexpを利用しましょう．

:::{hint}

2の冪乗でPerplexityを計算する場合は，この値の最大値は語彙数になるはずです．

:::

[問題 h-3] Perplexityを求める関数を作成してください．


In [21]:
def perplexity(pred_proba, bow):
    ppl = torch.exp(cross_entropy(pred_proba, bow)*bow.shape[0]/bow.sum())
    return ppl

In [22]:
ppl = perplexity(p, x)
print(f"Perplexity: {ppl.item()}")

Perplexity: 1108.4801025390625


### 訓練

ここまでで作成したVAEはNVDM (Neural Variational Document Model){cite}`Miao2016-xh`と呼ばれています．

[問題 h-4] 自作の訓練スクリプトやskorchを利用して，VAE（NVDM）を訓練してください．

hyper-paramsの設定例:
- epoch: 100
- batch_size: 128
- learning_rate: 0.01
- optimizer: Adam
- latent_dim: 20


In [23]:
# データの準備
train_data, test_data = train_test_split(docs_tr, test_size=0.2, random_state=42)
train_loader = DataLoader(TensorDataset(train_data), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(test_data), batch_size=32)

In [24]:
def train_vae(model, train_loader, test_loader, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            _, recon, mu, logvar = model(batch[0])
            ce_loss = cross_entropy(recon, batch[0])
            kl_loss = kld(mu, logvar)
            loss = ce_loss + kl_loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}")

        model.eval()
        with torch.no_grad():
          train_ppl = perplexity(model(train_data)[1], train_data)
          test_ppl = perplexity(model(test_data)[1], test_data)
        print(f"Training Perplexity: {train_ppl:.4f}")
        print(f"Test Perplexity: {test_ppl:.4f}")

[問題 h-5] 訓練終了時のtraining data全体とtest data全体に対するPerplexityをそれぞれ求めてください．

```python
model.eval()
with torch.no_grad():
    train_ppl = perplexity(model(train_data)[1], train_data)
    test_ppl = perplexity(model(test_data)[1], test_data)

print(f"Training Perplexity: {train_ppl:.4f}")
print(f"Test Perplexity: {test_ppl:.4f}")
```

### 類似単語検索

decoderのweightは潜在次元数$\times$語彙数 の行列になっています． shopeが逆ならば転置してください．これを（正規されていませんが，）トピックごとの単語分布だと見ることが可能です．word2vecの実装で登場した類似単語検索の関数にこのVAEが獲得したトピック-単語行列を適用し，適当な単語をクエリとして実験してみてください．

In [25]:
topic_word_distribution = ae.decoder[0].weight.T
print(f"{topic_word_distribution.shape=}")

topic_word_distribution = topic_word_distribution.detach().cpu().numpy()

topic_word_distribution.shape=torch.Size([20, 1000])


### 可視化

トピックの単語分布の中で，特に大きい値を持っている単語は「そのトピックでよく出現した単語」であると考えられます．これを上位n個ずつ表示することで，そのトピックが何について話されたものなのかを想像することができそうです．

In [26]:
import numpy as np

def print_topics(topic_word_distribution, id2word:list[str], topn:int=5):
    for topic_number, each_word_distribution in enumerate(topic_word_distribution):
        sorted_index = np.argsort(each_word_distribution)[::-1]
        print_message = " ".join([id2word[index] for index in sorted_index[:topn]])
        print(f"topic {topic_number:02}:\t", print_message)

In [27]:
id2word = list(word2id.keys())
print_topics(topic_word_distribution, id2word)

topic 00:	 thinking 25 ideas section start
topic 01:	 package cards 15 60 images
topic 02:	 distribution voice extra jews according
topic 03:	 response school worth getting heard
topic 04:	 send trade received basic chips
topic 05:	 community good market nature women
topic 06:	 available basically north currently algorithm
topic 07:	 took ax context break death
topic 08:	 products area gives hot number
topic 09:	 court supposed explain gun later
topic 10:	 engine consider advance legal model
topic 11:	 values day solution need carry
topic 12:	 statement digital true david groups
topic 13:	 open possibly end involved cs
topic 14:	 heard keyboard necessary saw book
topic 15:	 division considered said gov board
topic 16:	 conference unfortunately ve love logic
topic 17:	 air doing company changes anybody
topic 18:	 hope ax programs evidence peace
topic 19:	 stop wrong base cx god


実際には，あまり意味のない単語列が表示されているかも知れません．これは以下の理由が考えられます．
- 数字やstopword，単語の原形化や見出し語化など前処理を適切に行なっていない
- 出現頻度は大きいが意味のない単語が多すぎる

これから意味のある単語列を表示するにはterm-scoreという指標で並び替えることを考えますが，これは調べてみてください．

## Gaussian Softmax Trickを使ったVAE（GSM）による文書モデリング

Gaussian Softmax Model{cite}`GSM`

GSMの気持ち：  
潜在ベクトルの仮定する事前分布は，「本当に標準正規分布でいいの？」→もっと良い確率分布がありそう→Dirichlet分布を使いたい→VAEでは使い辛い→ガウス分布を使って近似しよう！

---
w.i.p.

これまでのVAEとの違い：
- zはGaussian Sampleそのままだったが，これに出力層がSoftmaxになっているMLPを適用する．

同じ空間上の点として「単語・文書・トピックの中心」を可視化するために，これらを埋め込む空間を用意します．
- decoderのweightを$k \times L$のトピック重心と，$v \times L$の単語埋め込みとの内積であるとする．
    - ここで$L$は単語埋め込みベクトルの次元数
    - 単語は単語埋め込みベクトル，トピックはトピック重心ベクトルをそれぞれ利用して二次元に圧縮した後に散布図にします．
    - 文書は$k$次元に圧縮されていますが，潜在表現ベクトルとトピック重心行列との内積を取ることで文書埋め込みベクトルが作成できます．あとは他のものと同様に二次元に圧縮して散布図にします．

[問題 h-6] $k$次元の潜在表現を入力として受け取り，$L$次元にするようなニューラルネットを作成してください（このネットワークをGeneratorと呼びましょう．）．ただし，簡単のために一層のNNであり，活性化関数はSoftmaxです．

In [28]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        return self.softmax(self.fc(x))

[問題 h-7] pytorchの[nn.Linearクラス](https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html#Linear)を修正し，単語埋め込み行列とトピック重心行列をパラメータとして持たせ，全結合層と同様の処理を行えるようにしてください．これをFactorizedLinearクラスとしてください．

- self.weightを削除
- nn.Parameterクラスを使い，新しいパラメータとしてword_embeddingsとtopic_centroidsを用意．
    - 初期化はnn.Linearのweightを参考にしてもいいですし，nn.Embeddingを参考にしても構いません．
- @propertyデコレータを使ってweightを返せるか検討して下さい
    - これが難しい場合はget_weightメソッドを定義して，weightを返してください．
- forwardメソッドで全結合層と同様の結果をもたらす処理を書いてください．


In [29]:
class FactorizedLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.word_embeddings = nn.Parameter(torch.Tensor(out_features, in_features))
        self.topic_centroids = nn.Parameter(torch.Tensor(in_features, in_features))

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.word_embeddings, a=np.sqrt(5))
        nn.init.kaiming_uniform_(self.topic_centroids, a=np.sqrt(5))

    @property
    def weight(self):
        return torch.matmul(self.word_embeddings, self.topic_centroids)

    def forward(self, input):
        return torch.matmul(input, self.weight.t())

[問題 h-8] GeneratorとFactorizedLinearをVAEクラスに導入し，GaussianSoftmaxクラスを作成してください．また，VAEと同様に訓練をおこなって，test setに対するPerplexityを表示してください．

In [30]:
class GaussianSoftmax(nn.Module):
    def __init__(self, in_features, n_components, n_topics):
        super().__init__()
        self.vae = VariationalAutoEncoder(in_features, n_components)
        self.generator = Generator(n_components, n_topics)
        self.factorized_linear = FactorizedLinear(n_topics, in_features)

    def forward(self, x):
        encoded, _, mu, logvar = self.vae(x)
        z = self.vae.reparameterization_trick(mu, logvar)
        topic_dist = self.generator(z)
        reconstructed = self.factorized_linear(topic_dist)
        return encoded, reconstructed, mu, logvar

In [31]:
# GaussianSoftmaxモデルの訓練
gs_model = GaussianSoftmax(vocab_size, 20, 20)
gs_optimizer = torch.optim.Adam(gs_model.parameters())

# 訓練の実行
train_vae(gs_model, train_loader, test_loader, gs_optimizer, num_epochs=5)  # エポック数を適宜調整してください

# テストデータに対するPerplexityの計算
gs_model.eval()
with torch.no_grad():
    test_ppl = perplexity(gs_model(test_data)[1], test_data)

print(f"GaussianSoftmax Test Perplexity: {test_ppl:.4f}")

Epoch 1/5, Loss: nan
Training Perplexity: nan
Test Perplexity: nan
Epoch 2/5, Loss: nan
Training Perplexity: nan
Test Perplexity: nan
Epoch 3/5, Loss: nan
Training Perplexity: nan
Test Perplexity: nan
Epoch 4/5, Loss: nan
Training Perplexity: nan
Test Perplexity: nan
Epoch 5/5, Loss: nan
Training Perplexity: nan
Test Perplexity: nan
GaussianSoftmax Test Perplexity: nan


### 類似単語検索

[問題 h-9] VAEの例を参考に類似単語検索を行って下さい．ただしこれまではdecoderのweightを利用していましたが，ここではFactorizedLinearインスタンスのword_embeddingsを使ってください．

In [32]:
def find_similar_words(word_embeddings, word2id, id2word, target_word, top_n=10):
    target_index = word2id[target_word]
    target_vector = word_embeddings[target_index]

    similarities = torch.cosine_similarity(target_vector.unsqueeze(0), word_embeddings)
    top_indices = torch.argsort(similarities, descending=True)[1:top_n+1] 

    similar_words = [(id2word[idx.item()], similarities[idx].item()) for idx in top_indices]
    return similar_words

### 可視化

[問題 h-10] 以下の手順で埋め込み空間の散布図を作成してください．

1. テストデータ全てに対する文書埋め込みベクトルを作成してください．
2. 以下の三種類のベクトルを，TSNEを用いて2次元に圧縮した後，散布図として表示してください．
    - 単語埋め込みベクトル
    - トピック重心ベクトル
    - 文書埋め込みベクトル
        - 全ての文書の埋め込みベクトルを使うと多すぎるので，以下の方法で代表的な文書を選択します：
            1. それぞれのトピックのトピック重心ベクトルと文書埋め込みベクトルとのコサイン類似度を計算
            2. 類似度が高い順に10個の文書をピックアップ（つまり20トピックならばそれぞれ10文書で200文書だけを利用します）

ただし，3種類のベクトルはそれぞれ別の色で表示し，可能であればそれぞれの点にラベルをつけてください．例えば：
    - "pen"という単語ならばこれが緑色のラベルとして点の近く or 点の代わりとして表示します．
    - トピック番号0~19については"【Topic 0】"のような赤色のラベルを使います．
    - 文書であれば先頭のいくつかの単語だけを青色でラベルとして利用します．
    


In [33]:
def visualize_embedding_space(word_embeddings, topic_centroids, doc_embeddings, id2word, selected_docs):
    all_embeddings = torch.cat([word_embeddings, topic_centroids, doc_embeddings], dim=0)

    tsne = TSNE(n_components=2, random_state=42)
    embeddings_2d = tsne.fit_transform(all_embeddings.detach().cpu().numpy())

    n_words = word_embeddings.shape[0]
    n_topics = topic_centroids.shape[0]
    words_2d = embeddings_2d[:n_words]
    topics_2d = embeddings_2d[n_words:n_words+n_topics]
    docs_2d = embeddings_2d[n_words+n_topics:]

    plt.figure(figsize=(20, 20))
    plt.scatter(words_2d[:, 0], words_2d[:, 1], c='green', alpha=0.5, label='Words')
    plt.scatter(topics_2d[:, 0], topics_2d[:, 1], c='red', alpha=0.5, label='Topics')
    plt.scatter(docs_2d[:, 0], docs_2d[:, 1], c='blue', alpha=0.5, label='Documents')

    for i in range(min(20, n_words)):
        plt.annotate(id2word[i], (words_2d[i, 0], words_2d[i, 1]), color='green', alpha=0.7)

    for i in range(n_topics):
        plt.annotate(f"Topic {i}", (topics_2d[i, 0], topics_2d[i, 1]), color='red', alpha=0.7)

    for i in range(min(20, len(selected_docs))):
        plt.annotate(f"Doc {i}", (docs_2d[i, 0], docs_2d[i, 1]), color='blue', alpha=0.7)

    plt.legend()
    plt.title("Embedding Space Visualization")
    plt.show()

In [34]:
# 使用例
gs_model.eval()
with torch.no_grad():
    doc_embeddings, _, _, _ = gs_model(test_data)

# 代表的な文書の選択（簡単のため、最初の200文書を使用）
selected_docs = [f"Doc {i}" for i in range(200)]

## 参考文献


```{bibliography}
:filter: docname in docnames
```